In [2]:
import pytest
import numpy as np
from modules.utils.services import embedder_services

# ==========================================
# 🧩 1️⃣ Kiểm tra Dense Embedding
# ==========================================
def test_encode_dense_basic():
    text = "Phân tích thị trường chứng khoán Việt Nam"
    dense_vecs = embedder_services.encode_dense(text)
    assert isinstance(dense_vecs, list), "Dense output phải là list"
    assert isinstance(dense_vecs[0], list), "Mỗi vector phải là list"
    assert len(dense_vecs[0]) == 384, f"Chiều dense vector phải = 384, hiện tại = {len(dense_vecs[0])}"
    print("\n✅ [Dense] OK - Chiều vector:", len(dense_vecs[0]))
    print("🔹 First 5 dims:", dense_vecs[0][:5])


# ==========================================
# 🧩 2️⃣ Kiểm tra Fit BM25
# ==========================================
def test_fit_bm25_and_vocab():
    corpus = [
        "Vingroup hỗ trợ đồng bào miền Trung",
        "Thị trường chứng khoán Việt Nam biến động",
        "Giá cổ phiếu VCB tăng mạnh hôm nay",
        "Techcombank và Vietcombank công bố kết quả kinh doanh quý 3"
    ]

    embedder_services.fit_bm25(corpus)
    assert embedder_services.bm25 is not None, "BM25 chưa được fit"
    assert len(embedder_services.vocab) > 0, "Từ vựng BM25 rỗng"
    print("\n✅ [BM25] OK - Vocab size:", len(embedder_services.vocab))


# ==========================================
# 🧩 3️⃣ Kiểm tra Sparse Embedding
# ==========================================
def test_encode_sparse_after_fit():
    text = "Phân tích cổ phiếu VCB"
    sparse_vecs = embedder_services.encode_sparse(text)
    assert isinstance(sparse_vecs, list), "Sparse output phải là list"
    assert "indices" in sparse_vecs[0] and "values" in sparse_vecs[0], "Cấu trúc sparse không hợp lệ"
    print("\n✅ [Sparse] OK - Số token trong vector:", len(sparse_vecs[0]['indices']))
    print("🔹 Sample indices:", sparse_vecs[0]['indices'][:5])
    print("🔹 Sample values:", sparse_vecs[0]['values'][:5])


# ==========================================
# 🧩 4️⃣ Kiểm tra Dense + Sparse song song
# ==========================================
def test_hybrid_encode():
    text = "Giá cổ phiếu VCB tăng mạnh"
    dense = embedder_services.encode_dense(text)
    sparse = embedder_services.encode_sparse(text)

    assert len(dense[0]) == 384, "Dense vector phải có 384 chiều"
    assert isinstance(sparse[0]['indices'], list), "Sparse indices phải là list"
    print("\n✅ [Hybrid] OK - Dense(384) + Sparse(length=", len(sparse[0]['indices']), ")")
    print("🔹 Dense first dims:", dense[0][:5])


# ==========================================
# 🧩 5️⃣ Kiểm tra lỗi nếu chưa fit BM25
# ==========================================
def test_sparse_before_fit():
    # Tạo instance mới chưa fit để test
    from modules.utils.services import EmbedderServices
    new_embedder = EmbedderServices(auto_fit=False)
    with pytest.raises(ValueError, match="BM25 chưa được fit"):
        new_embedder.encode_sparse("cổ phiếu VCB")
    print("\n✅ [Error Handling] OK - Cảnh báo chưa fit BM25 hoạt động đúng")
if __name__ == "__main__":
    test_encode_dense_basic()
    test_fit_bm25_and_vocab()
    test_encode_sparse_after_fit()
    test_hybrid_encode()
    test_sparse_before_fit()


✅ [Dense] OK - Chiều vector: 384
🔹 First 5 dims: [-0.42621153593063354, 0.4696739912033081, -0.08442647755146027, -0.06147570535540581, -0.23391616344451904]

✅ [BM25] OK - Vocab size: 34

✅ [Sparse] OK - Số token trong vector: 3
🔹 Sample indices: [6, 15, 25]
🔹 Sample values: [0.8472978603872037, 0.8472978603872037, 0.8472978603872037]

✅ [Hybrid] OK - Dense(384) + Sparse(length= 6 )
🔹 Dense first dims: [-0.3635924756526947, 0.16256432235240936, -0.22329771518707275, 0.07123153656721115, -0.05664300546050072]
EmbedderServices device: cuda

✅ [Error Handling] OK - Cảnh báo chưa fit BM25 hoạt động đúng


In [3]:
from modules.core.state import GlobalState
from modules.nodes.embedder import embed_query
import numpy as np

def test_embedder():
    print("=== 🧩 TEST EMBEDDER MODULE (rag & hybrid) ===")

    for route_type in ["rag", "hybrid"]:
        print(f"\n🔹 TEST ROUTE: {route_type.upper()}")
        state = GlobalState(user_query="Phân tích cổ phiếu TCB", route_to=route_type)

        state = embed_query(state)

        if state.query_embedding is None:
            print("❌ Không tạo được embedding.")
            print("Debug info:", state.debug_info)
            continue

        dense_vec = state.query_embedding.get("dense_vector")
        sparse_vec = state.query_embedding.get("sparse_vector")

        print("✅ Embedding thành công.")
        print(f"🧠 Dense vector length: {len(dense_vec) if dense_vec is not None else 'None'}")
        if isinstance(dense_vec, (list, np.ndarray)):
            print(f"🔹 First 5 dims: {dense_vec[:5]}")
        else:
            print("⚠️ Dense vector không phải list hoặc np.ndarray")

        print(f"🌾 Sparse vector: {'OK' if sparse_vec else 'None'}")
        print(f"📜 Debug Info: {state.debug_info}")
        print("LLM status:", state.llm_status)
        print("Embed route:", state.debug_info.get('embed_route'))

if __name__ == "__main__":
    test_embedder()


=== 🧩 TEST EMBEDDER MODULE (rag & hybrid) ===

🔹 TEST ROUTE: RAG
✅ Embedding thành công.
🧠 Dense vector length: 384
🔹 First 5 dims: [-0.5979602932929993, 0.3590678870677948, 0.09738045930862427, -0.21666592359542847, -0.21862435340881348]
🌾 Sparse vector: OK
📜 Debug Info: {'embed_dense_dim': 384, 'embed_sparse_status': 'ok', 'embed_status': 'success', 'embed_query_text': 'Phân tích cổ phiếu TCB', 'embed_route': 'rag'}
LLM status: embed_success
Embed route: rag

🔹 TEST ROUTE: HYBRID
✅ Embedding thành công.
🧠 Dense vector length: 384
🔹 First 5 dims: [-0.5979602932929993, 0.3590678870677948, 0.09738045930862427, -0.21666592359542847, -0.21862435340881348]
🌾 Sparse vector: OK
📜 Debug Info: {'embed_dense_dim': 384, 'embed_sparse_status': 'ok', 'embed_status': 'success', 'embed_query_text': 'Phân tích cổ phiếu TCB', 'embed_route': 'hybrid'}
LLM status: embed_success
Embed route: hybrid


In [4]:
from modules.core.state import GlobalState
from modules.nodes.embedder import embed_query
from modules.nodes.vector_db import search_vector_db


def test_vector_db_realdata():
    """
    🧩 TEST VECTOR_DB TRÊN DỮ LIỆU THẬT (CAFÉF)
    ------------------------------------------------
    ✅ Tự động chạy embedding + search
    ✅ Kiểm tra cả route RAG và HYBRID
    ✅ Hiển thị debug info + kết quả an toàn
    """
    print("\n=== 🧩 TEST VECTOR_DB TRÊN DỮ LIỆU THẬT (CAFÉF) ===")

    query = "Phân tích cổ phiếu HPG hôm nay"
    state = GlobalState(user_query=query, debug=True)

    for route in ["rag", "hybrid"]:
        print(f"\n🔹 TEST ROUTE: {route.upper()}")
        state.route_to = route

        try:
            # --- Bước 1: Embedding
            state = embed_query(state)
            if not state.query_embedding:
                print("❌ Không tạo được embedding.")
                print("Debug info:", getattr(state, "debug_info", {}))
                continue

            dense_dim = len(state.query_embedding.get("dense_vector", []))
            print(f"✅ [Dense] OK - Chiều vector: {dense_dim}")

            sparse_vec = state.query_embedding.get("sparse_vector")
            if sparse_vec:
                print(f"✅ [Sparse] OK - Số token trong vector: {len(sparse_vec[0]['indices'])}")
            else:
                print("⚠️ [Sparse] Chưa có dữ liệu (BM25 chưa fit?)")

            # --- Bước 2: Gọi Vector DB
            state = search_vector_db(state, top_k=5, alpha=0.7)

            # --- Bước 3: In kết quả an toàn
            print(f"\n=== 🔍 Kết quả tìm kiếm cho truy vấn: “{query}” ===")
            if not state.search_results:
                print("⚠️ Không tìm thấy kết quả nào trong Qdrant.")
            else:
                for idx, doc in enumerate(state.search_results, 1):
                    title = doc.get("title", "❌ Không có tiêu đề")
                    time = doc.get("time", "❌ Không rõ thời gian")
                    snippet = doc.get("content", "") or "(Không có nội dung trích dẫn)"
                    print(f"{idx}. [{doc.get('score', 0):.4f}] {title} — {time}")
                    print(f"   {snippet}\n")

            # --- Bước 4: Debug Info
            print("📜 Debug Info:", getattr(state, "debug_info", {}))
            print("🧠 LLM status:", getattr(state, "llm_status", "undefined"))

        except Exception as e:
            print(f"❌ Lỗi trong route {route}: {e}")

    print("\n✅ [Test] Hoàn tất kiểm thử Vector DB.")

if __name__ == "__main__":
    test_vector_db_realdata()


=== 🧩 TEST VECTOR_DB TRÊN DỮ LIỆU THẬT (CAFÉF) ===

🔹 TEST ROUTE: RAG
✅ [Dense] OK - Chiều vector: 384
✅ [Sparse] OK - Số token trong vector: 4
[FusionSearch] Found 10 results
GS thắng giải thưởng của tỷ phú Phạm Nhật Vượng vừa được Nobel vinh danh: "Tôi có một tuổi thơ gian khó" | score=0.75 | time_ts=1759998000
MBS hoàn tất phát hành 17,18 triệu cổ phiếu thưởng cho cổ đông | score=0.75 | time_ts=1759994940
MBS hoàn tất phát hành 17,18 triệu cổ phiếu thưởng cho cổ đông | score=0.6667 | time_ts=1759994940

=== 🔍 Kết quả tìm kiếm cho truy vấn: “Phân tích cổ phiếu HPG hôm nay” ===
1. [0.7500] GS thắng giải thưởng của tỷ phú Phạm Nhật Vượng vừa được Nobel vinh danh: "Tôi có một tuổi thơ gian khó" — 09-10-2025 15:20:00
   loại bỏ chất ô nhiễm trong nước... Bằng cách thay đổi các "viên gạch" cấu thành MOFs, các nhà hóa học có thể thiết kế vật liệu để hấp thụ và lưu trữ các chất cụ thể. Ngoài ra, MOFs cũng có thể thúc đẩy các phản ứng hóa học hoặc dẫn điện. Chủ nhân của giải thưởng Nobel Hó

In [4]:
from modules.core.state import GlobalState
from modules.nodes.processor import processor_query   # ⚠️ đổi nếu là function
from modules.nodes.router import route_intent
from modules.nodes.embedder import embed_query
from modules.nodes.vector_db import search_vector_db
from modules.nodes.retriever import retrieve_documents
from modules.api import stock_api
import pytz
from datetime import datetime

def test_rag_pipeline_realdata():
    print("\n=== 🧩 TEST FULL RAG PIPELINE TRÊN DỮ LIỆU THẬT ===\n")

    # 1️⃣ Khởi tạo state
    state = GlobalState()
    state.debug = True
    state.user_query = "Hôm nay có tin tức gì mới?"

    # 2️⃣ Processor
    state = processor_query(state)   # ✅ sửa ở đây
    print(f"🧠 [Processor] query_clean = {state.processed_query}")
    print(f"🧭 route_to = {state.route_to}")

    # 3️⃣ Router
    state = route_intent(state)
    print(f"➡️ [Router] route_to = {state.route_to}, api_type = {state.api_type}")

    # 4️⃣ API
    if getattr(state, "api_type", None) == "stock":
        state.api_response = stock_api.format_stock_info("TCB")
        print(f"📊 [API] response = {state.api_response}")

    # 5️⃣ Embedder
    state = embed_query(state)
    print(f"🧩 [Embedder] Dense dim = {len(state.query_embedding['dense_vector']) if state.query_embedding else 0}")

    # 6️⃣ Vector DB
    state = search_vector_db(state, top_k=5, alpha=0.7)
    print(f"🔍 [VectorDB] Found {len(state.search_results)} results (llm_status={state.llm_status})")

    # 7️⃣ Retriever
    state = retrieve_documents(state, max_chars=1500)
    print(f"📄 [Retriever] Retrieved {len(state.retrieved_docs)} docs, context_len={len(state.context)}")

    print("\n✅ [Test] Hoàn tất kiểm thử full pipeline.\n")

if __name__ == "__main__":
    test_rag_pipeline_realdata()



=== 🧩 TEST FULL RAG PIPELINE TRÊN DỮ LIỆU THẬT ===

🧠 [Processor] query_clean = hôm nay có tin tức gì mới
🧭 route_to = None
➡️ [Router] route_to = rag, api_type = None
🧩 [Embedder] Dense dim = 384
[FusionSearch] Found 8 results
Cập nhật số liệu CTCK ngày 10/10: Những tên tuổi đầu tiên hé lộ KQKD quý 3 với nhiều khoản lãi "tăng bằng lần" | score=0.75 | time_ts=1760030400
Chuyên gia Dragon Capital: “Đừng ngạc nhiên, thắc mắc hay sợ hãi khi thấy chứng khoán có nhịp giảm 5-10%” | score=0.75 | time_ts=1760051520
Chuyên gia Dragon Capital: “Đừng ngạc nhiên, thắc mắc hay sợ hãi khi thấy chứng khoán có nhịp giảm 5-10%” | score=0.6667 | time_ts=1760051520
🔍 [VectorDB] Found 8 results (llm_status=vector_db_success)
*** RETRIEVED DOCS ***
{'id': '599e18f9-7df6-12f2-eea3-1241091ed4ba', 'score': 0.75, 'title': 'Cập nhật số liệu CTCK ngày 10/10: Những tên tuổi đầu tiên hé lộ KQKD quý 3 với nhiều khoản lãi "tăng bằng lần"', 'time': '10-10-2025 00:20:00', 'time_ts': 1760030400, 'url': '', 'content_pr

In [2]:
from modules.utils.services import qdrant_services

points, _ = qdrant_services.client.scroll(
    collection_name="cafef_articles", 
    with_payload=True, 
    limit=5
)
for p in points:
    print(p.payload)


{'id': '003f5a82fc72362416a776092d1120a3', 'title': "Thủ tướng đề nghị doanh nghiệp, doanh nhân Việt Nam phát huy '3 tiên phong'", 'time': '09-10-2025 19:13:00', 'time_ts': 1760011980, 'summary': 'Trong bối cảnh lần đầu tiên số doanh nghiệp đang hoạt động vượt mốc 1 triệu, Thủ tướng Phạm Minh Chính nêu 5 mong muốn và đề nghị các doanh nghiệp, doanh nhân Việt Nam phát huy "3 tiên phong", khai thác các không gian phát triển mới, "vươn xa ra biển lớn, tiến sâu vào lòng đất và bay cao lên vũ trụ", góp phần đưa đất nước phát triển nhanh và bền vững, vững bước đi lên chủ nghĩa xã hội, bước vào kỷ nguyên mới, phát triển giàu mạnh, thịnh vượng, văn minh, phồn vinh', 'url': 'https://cafef.vn/thu-tuong-de-nghi-doanh-nghiep-doanh-nhan-viet-nam-phat-huy-3-tien-phong-188251009182707694.chn', 'content': 'phủ cam kết đồng hành cùng với doanh nghiệp, doanh nhân trên tinh thần là kiến tạo, liêm chính, hành động, phục vụ nhân dân, trong đó có doanh nghiệp và doanh nhân. Người đứng đầu Chính phủ cũng đề 

In [5]:
from modules.nodes.prompt_builder import build_prompt
from modules.nodes.retriever import retrieve_documents
from modules.core.state import GlobalState
from modules.nodes.processor import processor_query
from modules.nodes.router import route_intent
from modules.api import stock_api
from modules.nodes.embedder import embed_query
from modules.nodes.vector_db import search_vector_db

def test_prompt_realdata():
    """
    🧩 TEST PROMPT BUILDER TRÊN DỮ LIỆU THẬT
    ------------------------------------------------
    ✅ Giả lập query → embedder → vector_db → retriever → prompt
    ✅ Kiểm tra prompt cuối cùng sinh ra có context + lịch sử hội thoại
    """
    print("\n=== 🧩 TEST PROMPT BUILDER TRÊN DỮ LIỆU THẬT ===")

    # 1️⃣ Khởi tạo state
    state = GlobalState()
    state.debug = True
    state.user_query = "Hôm nay có tin tức gì mới?"

    # 2️⃣ Processor
    state = processor_query(state)   # ✅ sửa ở đây
    print(f"🧠 [Processor] query_clean = {state.processed_query}")
    print(f"🧭 route_to = {state.route_to}")

    # 3️⃣ Router
    state = route_intent(state)
    print(f"➡️ [Router] route_to = {state.route_to}, api_type = {state.api_type}")

    # 4️⃣ API
    if getattr(state, "api_type", None) == "stock":
        state.api_response = stock_api.format_stock_info("TCB")
        print(f"📊 [API] response = {state.api_response}")

    # 5️⃣ Embedder
    state = embed_query(state)
    print(f"🧩 [Embedder] Dense dim = {len(state.query_embedding['dense_vector']) if state.query_embedding else 0}")

    # 6️⃣ Vector DB
    state = search_vector_db(state, top_k=5, alpha=0.7)
    print(f"🔍 [VectorDB] Found {len(state.search_results)} results (llm_status={state.llm_status})")

    # 7️⃣ Retriever
    state = retrieve_documents(state, max_chars=1500)
    print(f"📄 [Retriever] Retrieved {len(state.retrieved_docs)} docs, context_len={len(state.context)}")

    # 4️⃣ Build prompt
    state = build_prompt(state, max_context_chars=1200)

    # 5️⃣ In preview
    print("\n=== 🧩 PROMPT SAMPLE (first 800 chars) ===")
    print(state.prompt[:800])
    print("\n---------------------------------------------")
    print(f"📄 Prompt length: {len(state.prompt)}")
    print(f"🧠 Context docs: {len(state.retrieved_docs)}")
    print(f"🔍 LLM status: {getattr(state, 'llm_status', 'undefined')}")

    print("\n✅ [Test] Hoàn tất kiểm thử PROMPT BUILDER.\n")

if __name__ == "__main__":
    test_prompt_realdata()



=== 🧩 TEST PROMPT BUILDER TRÊN DỮ LIỆU THẬT ===
🧠 [Processor] query_clean = hôm nay có tin tức gì mới
🧭 route_to = None
➡️ [Router] route_to = rag, api_type = None
🧩 [Embedder] Dense dim = 384
[FusionSearch] Found 8 results
Cập nhật số liệu CTCK ngày 10/10: Những tên tuổi đầu tiên hé lộ KQKD quý 3 với nhiều khoản lãi "tăng bằng lần" | score=0.75 | time_ts=1760030400
Chuyên gia Dragon Capital: “Đừng ngạc nhiên, thắc mắc hay sợ hãi khi thấy chứng khoán có nhịp giảm 5-10%” | score=0.75 | time_ts=1760051520
Cập nhật số liệu CTCK ngày 10/10: Những tên tuổi đầu tiên hé lộ KQKD quý 3 với nhiều khoản lãi "tăng bằng lần" | score=0.6667 | time_ts=1760030400
🔍 [VectorDB] Found 8 results (llm_status=vector_db_success)
*** RETRIEVED DOCS ***
{'id': '599e18f9-7df6-12f2-eea3-1241091ed4ba', 'score': 0.75, 'title': 'Cập nhật số liệu CTCK ngày 10/10: Những tên tuổi đầu tiên hé lộ KQKD quý 3 với nhiều khoản lãi "tăng bằng lần"', 'time': '10-10-2025 00:20:00', 'time_ts': 1760030400, 'url': '', 'content_p

In [3]:
from modules.nodes.response_generator import response_node  # import file chứa response_node
from modules.core.state import GlobalState
from modules.nodes.processor import processor_query
from modules.nodes.router import route_intent
from modules.api import stock_api
from modules.nodes.embedder import embed_query
from modules.nodes.vector_db import search_vector_db
from modules.nodes.retriever import retrieve_documents
from modules.nodes.prompt_builder import build_prompt

def test_full_rag_pipeline_realdata():
    """
    🧩 TEST FULL RAG PIPELINE TRÊN DỮ LIỆU THẬT (END-TO-END)
    ----------------------------------------------------------
    Pipeline:
      user_query → processor → router → api → embedder → vector_db
      → retriever → prompt_builder → response_node (LLM)
    ✅ Kiểm tra toàn bộ luồng RAG thật từ query đến phản hồi.
    """
    print("\n=== 🧩 TEST FULL RAG PIPELINE TRÊN DỮ LIỆU THẬT ===")

    # 1️⃣ Khởi tạo state
    state = GlobalState(user_query="Giá cổ phiếu VCB hôm nay?", debug=True)

    # 2️⃣ Processor
    state = processor_query(state)
    print(f"🧠 [Processor] query_clean = {state.processed_query}")

    # 3️⃣ Router
    state = route_intent(state)
    print(f"🧭 [Router] route_to = {state.route_to}, api_type = {state.api_type}")

    # 4️⃣ API (nếu cần)
    if getattr(state, "api_type", None) == "stock":
        state.api_response = stock_api.format_stock_info("VCB")
        print(f"📊 [API] response = {state.api_response}")

    # 5️⃣ Embedder
    state = embed_query(state)
    print(f"🧩 [Embedder] Dense dim = {len(state.query_embedding['dense_vector']) if state.query_embedding else 0}")

    # 6️⃣ Vector DB
    state = search_vector_db(state, top_k=10, alpha=0.7)
    print(f"🔍 [VectorDB] Found {len(state.search_results)} results (llm_status={state.llm_status})")

    # 7️⃣ Retriever
    state = retrieve_documents(state, max_chars=1500)
    print(f"📄 [Retriever] Retrieved {len(state.retrieved_docs)} docs, context_len={len(state.context)}")

    # 8️⃣ Prompt Builder
    state = build_prompt(state, max_context_chars=1200)
    print(f"📜 [Prompt] Built successfully, length={len(state.prompt)}")

    # 9️⃣ Response Generator (LLM)
    state = response_node(state)
    print(f"🤖 [LLM] Response length={len(state.final_answer)}")
    print(f"🧾 [Final Answer]:\n{state.final_answer[:800]}")

    # 10️⃣ Debug Info Summary
    print("\n=== 🧩 DEBUG SUMMARY ===")
    for k, v in getattr(state, "debug_info", {}).items():
        print(f"{k}: {v}")
    print("✅ [Test] Hoàn tất kiểm thử full RAG pipeline.\n")

if __name__ == "__main__":
    test_full_rag_pipeline_realdata()


2025-10-11 18:42:10,302 [DEBUG] RAGPipeline: router: API
2025-10-11 18:42:10,303 [DEBUG] RAGPipeline: intent: stock



=== 🧩 TEST FULL RAG PIPELINE TRÊN DỮ LIỆU THẬT ===
🧠 [Processor] query_clean = giá cổ phiếu vcb hôm nay
🧭 [Router] route_to = api, api_type = stock
📊 [API] response = 📊 **VCB** — Giá hiện tại: 64,200.0 VNĐ
• Mở cửa: 63,800.0 | Cao nhất: 68,200.0 | Thấp nhất: 59,400.0
• Thay đổi: +400.00 (+0.63%)
• Khối lượng: 4,985,600
🕒 Cập nhật: 11-10-2025 18:41:40
🧩 [Embedder] Dense dim = 0
🔍 [VectorDB] Found 0 results (llm_status=vector_db_skipped)
📄 [Retriever] Retrieved 0 docs, context_len=0
📜 [Prompt] Built successfully, length=0
🤖 [LLM] Response length=185
🧾 [Final Answer]:
📊 **VCB** — Giá hiện tại: 64,200.0 VNĐ
• Mở cửa: 63,800.0 | Cao nhất: 68,200.0 | Thấp nhất: 59,400.0
• Thay đổi: +400.00 (+0.63%)
• Khối lượng: 4,985,600
🕒 Cập nhật: 11-10-2025 18:41:40

=== 🧩 DEBUG SUMMARY ===
route: stock_api
embed_status: Skipped
vector_db: Skipped
retriever: Skipped
prompt_builder: Skipped
final_answer: 📊 **VCB** — Giá hiện tại: 64,200.0 VNĐ
• Mở cửa: 63,800.0 | Cao nhất: 68,200.0 | Thấp nhất: 59,400.0
